Setup

In [3]:
from tensorflow_docs.vis import embed

from tensorflow import keras
from keras import layers

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

Data preparation

In [16]:
MAX_SEQ_LENGTH = 40
NUM_FEATURES = 1024
IMG_SIZE = 128 #320x240 la resolucion de los videos

INDEX = 4

EPOCHS = 5

Preparar csv's

In [19]:
center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze() #Utiliza squeeze() para eliminar las dimensiones adicionales, si las hubiera, y devuelve el arreglo resultante.
    return cropped

def MostrarImagen(frame):
    cv2.imshow('Frame', frame)
    cv2.waitKey(0)  # Esperar hasta que se presione una tecla
    cv2.destroyAllWindows()

def IsBlack(frame, y, x):
    height, width, _ = frame.shape
    cond = True
    r = 2
    i = -r
    j = -r
    while i < r and i + y < height and i + y > 0 and cond:
        j = 0
        while j < r and j + x < width and j + x > 0 and cond:
            if frame[y+i, x+j].any() != 0:
                cond = False
            j = j + 1
        i = i + 1
              
    return cond
    #b, g, r = frame[y, x]
    #if b != 0 or g != 0 or r != 0:
    #      return False
    #return True

def EncontrarCorte(frame):
    height, width, _ = frame.shape

    halfWidth = int(width / 2)
    halfHeight = int(height / 2)

    up = 0
    down = height - 1
    left = halfWidth - 1
    right = halfWidth + 1

    while IsBlack(frame, up, halfWidth) and up < halfHeight:
            up = up + 1
    while IsBlack(frame, down, halfWidth) and down > halfHeight:
            down = down - 1

    while not IsBlack(frame, up + 5, left) and left > 0:
            left = left - 1
    while not IsBlack(frame, up + 5, right) and right < width:
            right = right + 1

    up = up - 1
    down = down + 1
    left = left - 1
    right = right + 1

    return right, up, left, down
    #return frame[up:down,left:right]
def CalcularDimension(path):
    cap = cv2.VideoCapture(path)
    try:
        #while True:
            ret, frame = cap.read()
            #if not ret:
            #    break
            #MostrarImagen(frame)
            right, up, left, down = EncontrarCorte(frame)
            #MostrarImagen(frame[up:down,left:right])
            #MostrarImagen(frame[179:716,291:1002])
            #nombre = path.split('\\')[1]
            #nombre = nombre + '_' + path.split('\\')[2].split('.')[0]
            #cv2.imwrite('ecografias_recortadas\\' + nombre +'.jpg', frame[up:down,left:right])
            #frame = crop_center(frame)
            MostrarImagen(frame[0:531,0:468])
            #frame = frame[:, :, [2, 1, 0]]
            #MostrarImagen(frame)
            #Noviembre2020\PAC48\post.mov,"[936, 176, 357, 723]",3.0
            #if len(frames) == max_frames:
            #    break
    finally:
        cap.release()
    return [right, up, left, down]
    
CalcularDimension('Noviembre2020\PAC23\POST.mov')

KeyboardInterrupt: 

In [4]:
# Ruta del directorio que deseas leer
dir = 'Noviembre2020'
df = pd.DataFrame({'Direccion':[], 'Dimension':[], 'Target':[]})

def MostrarFuncion(df, directorio):
    for archivo in os.listdir(directorio):
        # Recorrer todos los archivos en el directorio
        ruta_completa = os.path.join(directorio, archivo)
        
        # Verificar si es un archivo
        if os.path.isdir(ruta_completa):
            df = MostrarFuncion(df, ruta_completa)
        if os.path.isfile(ruta_completa) and ruta_completa.split('.')[-1] == 'mov':
            nueva_fila = {'Direccion':ruta_completa, 'Dimension':CalcularDimension(ruta_completa), 'Target' : 3}
            #df = pd.concat([pd.DataFrame(nueva_fila), df], ignore_index = False)
            df = df.append(nueva_fila, ignore_index=True)
            #print(archivo)
    return df

#df = pd.DataFrame(columns=['Direccion', 'Dimension', 'Target'])
display(df)
df = MostrarFuncion(df, dir)
df.to_csv('lista_videos.csv', index=False)

,Direccion,Dimension,Target


C:\Users\Francisco\AppData\Local\Temp\ipykernel_10516\2473693984.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_10516\2473693984.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_10516\2473693984.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_10516\2473693984.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_

KeyboardInterrupt: 

In [62]:
list_df = pd.read_csv("video list/_lista_videos.csv")
list_df = list_df.sample(frac=1) 
list_df.head()

cantidad = len(list_df)
cantidad_train = int(cantidad * 0.8)
cantidad_test = cantidad - cantidad_train

datos_train = list_df[:cantidad_train]
datos_test = list_df[cantidad_train:]

datos_train.to_csv("video list/train.csv", index=False)
datos_test.to_csv("video list/test.csv", index=False)

datos_train.head()
datos_test.head()


,Direccion,Dimension,Target
27,Noviembre2020\PAC14\PRE.mov,"[1011, 178, 289, 723]",3.0
32,Noviembre2020\PAC17\POST.mov,"[1009, 176, 285, 723]",0.0
33,Noviembre2020\PAC17\PRE.mov,"[1009, 174, 285, 723]",3.0
15,Noviembre2020\PAC08\PRE.mov,"[933, 176, 357, 724]",3.0
28,Noviembre2020\PAC15\POST.mov,"[938, 176, 355, 712]",2.0


Define hyperparameters

In [3]:
train_df = pd.read_csv("video list/train.csv")
test_df = pd.read_csv("video list/test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)

#----------------------------------------------------------------------------------------------

def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze() #Utiliza squeeze() para eliminar las dimensiones adicionales, si las hubiera, y devuelve el arreglo resultante.
    return cropped

#----------------------------------------------------------------------------------------------

# Following method is modified from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def get_dimension(res):
    res = res[1:-1].split(', ')
    res.reverse()

    right = int(res[0])
    up = int(res[1])
    left = int(res[2])
    down = int(res[3])
    return right, up, left, down


def load_video(path, resolution, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    #Se obtiene la resolucion de la img
    right, up, left, down = get_dimension(resolution)
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            #frame = frame[up:down, left:right]

            frame = frame[up:down, left:right]
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

#----------------------------------------------------------------------------------------------

def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg", #significa que se aplica average pooling para reducir la dimensionalidad de las características extraídas antes de pasarlas
        # a la capa de clasificación final. Esta capa de promediado se utiliza para obtener una representación global de las características y 
        # reducir el impacto de las ubicaciones específicas en la clasificación final, lo que puede ayudar a mejorar la generalización del modelo.
        input_shape=(IMG_SIZE, IMG_SIZE, 3), #CNN preentrenada para la clasificación de imágenes. DenseNet es una arquitectura de red neuronal 
        # que se destaca por su conexión densa, donde cada capa tiene conexiones directas con todas las capas anteriores. El número "121" en 
        # DenseNet121 indica que este modelo específico tiene 121 capas en total.
    )
    preprocess_input = keras.applications.densenet.preprocess_input
    # Convierte la imagen de entrada en un arreglo NumPy.
    # Normaliza el arreglo de la imagen dividiendo cada píxel por 255.0 para escalar los valores entre 0 y 1.
    # Utiliza una transformación específica de DenseNet que involucra el reordenamiento de los canales de color. En concreto, cambia el 
    # orden de los canales de color de RGB (Rojo, Verde, Azul) a BGR (Azul, Verde, Rojo). Esto se debe a que los modelos DenseNet generalmente 
    # se entrenan con imágenes en el esquema BGR.

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

# Label preprocessing with StringLookup.
#label_processor = keras.layers.StringLookup(
#    num_oov_indices=0, vocabulary=np.unique(train_df["Target"]), mask_token=None
#)
#num_oov_indices: Especifica el número de índices "fuera del vocabulario" (Out-of-Vocabulary, OOV). En este caso, se establece en 0, lo que significa que no se permiten valores OOV.
#vocabulary: El vocabulario utilizado para codificar las etiquetas. Se obtiene a partir de los valores únicos en la columna "Target" del dataframe train_df.
#mask_token: Especifica un token de máscara opcional. En este caso, se establece en None, lo que significa que no se utiliza un token de máscara.
#print(label_processor.get_vocabulary())
print(np.unique(train_df["Target"]))

def prepare_all_videos(df):
    
    num_samples = len(df)
    video_paths = df["Direccion"].values.tolist()
    video_resolution = df["Dimension"].values.tolist()
    labels = df["Target"].values
    labels = np.reshape(labels, (len(labels), 1))

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        #shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        shape=(0, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        #print(str(idx) + ' de ' + str(num_samples))
        frames = load_video(path, video_resolution[idx])

        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate(frames, padding)

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels

Total videos for training: 80
Total videos for testing: 20
[0. 1. 2. 3.]


In [4]:
def prepare_all_videos(df):
    
    num_samples = len(df)
    video_paths = df["Direccion"].values.tolist()
    video_resolution = df["Dimension"].values.tolist()
    labels = df["Target"].values
    labels = np.reshape(labels, (len(labels), 1))

    shortLabel = np.zeros(
        shape=(1, 1), dtype="float32"
    )
    finalLabels = np.zeros(
        shape=(0, 1), dtype="float32"
    )

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        #shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        shape=(0, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    length = len(video_paths)
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        #print(str(idx) + ' de ' + str(num_samples))
        inf = 0
        print("IDX: " + str(idx) + "/" + str(length) + "| path: " + path + "|inf:" + str(inf))
        video = load_video(path, video_resolution[idx])
        while inf < len(video):
            print("inf: " + str(inf) + "|to: " + str(inf + MAX_SEQ_LENGTH))
            frames = video[inf:inf + MAX_SEQ_LENGTH]
            inf = inf + MAX_SEQ_LENGTH
            # Pad shorter videos.
            if len(frames) < MAX_SEQ_LENGTH:                
                diff = MAX_SEQ_LENGTH - len(frames)
                padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
                frames = np.concatenate((frames, padding), axis=0)

            frames = frames[None, ...]

            # Initialize placeholder to store the features of the current video.
            temp_frame_features = np.zeros(
                shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
            )

            # Extract features from the frames of the current video.
            for i, batch in enumerate(frames):
                video_length = batch.shape[0]
                length = min(MAX_SEQ_LENGTH, video_length)
                for j in range(length):
                    if np.mean(batch[j, :]) > 0.0:
                        temp_frame_features[i, j, :] = feature_extractor.predict(
                            batch[None, j, :]
                        )

                    else:
                        temp_frame_features[i, j, :] = 0.0

            #frame_features[idx,] = temp_frame_features.squeeze()
            frame_features = np.concatenate((frame_features, temp_frame_features), axis=0)
            frame_features[idx,] = temp_frame_features.squeeze()
            
            shortLabel[0,0] = labels[idx, 0]
            finalLabels = np.concatenate((finalLabels, shortLabel), axis = 0)
    return frame_features, finalLabels

Prepare all videos (5m 10.1s)

In [70]:
train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

np.save(f"Index{INDEX}/train_data.npy", train_data, fix_imports=True, allow_pickle=False)
np.save(f"Index{INDEX}/train_labels.npy", train_labels, fix_imports=True, allow_pickle=False)
np.save(f"Index{INDEX}/test_data.npy", test_data, fix_imports=True, allow_pickle=False)
np.save(f"Index{INDEX}/test_labels.npy", test_labels, fix_imports=True, allow_pickle=False)

IDX: 0/80| path: Noviembre2020\PAC32\pre (convertido).mov|inf:0
inf: 0|to: 40
1/1 [==============================] - 0s 42ms/step
inf: 40|to: 80
1/1 [==============================] - 0s 44ms/step
inf: 80|to: 120
1/1 [==============================] - 0s 40ms/step
inf: 120|to: 160
1/1 [==============================] - 0s 41ms/step
inf: 160|to: 200
1/1 [==============================] - 0s 45ms/step
inf: 200|to: 240
1/1 [==============================] - 0s 44ms/step
inf: 240|to: 280
1/1 [==============================] - 0s 478ms/step
inf: 280|to: 320
1/1 [==============================] - 0s 46ms/step
inf: 320|to: 360
1/1 [==============================] - 0s 220ms/step
inf: 360|to: 400
1/1 [==============================] - 0s 44ms/step
IDX: 1/40| path: Noviembre2020\PAC35\post.mov|inf:0
inf: 0|to: 40
1/1 [==============================] - 0s 41ms/step
inf: 40|to: 80
1/1 [==============================] - 0s 46ms/step
inf: 80|to: 120
1/1 [==============================] - 0s 64ms/st

In [17]:
train_data, train_labels = np.load(f"Index{INDEX}/_features/train_data.npy"), np.load(f"Index{INDEX}/_features/train_labels.npy")
test_data, test_labels = np.load(f"Index{INDEX}/_features/test_data.npy"), np.load(f"Index{INDEX}/_features/test_labels.npy")

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

(580, 40, 1024)
(580, 1)
(141, 40, 1024)
(141, 1)


Modificar Datos

In [ ]:
column = train_labels[:, 0]
column[column < 3] = 0
column[column >= 3] = 1

column = test_labels[:, 0]
column[column < 3] = 0
column[column >= 3] = 1

print(f"Frame features in train set: {train_data.shape}")

Frame features in train set: (580, 40, 1024)


Expandir Datos

In [6]:
def ArrayExpand(tensor):
    count = tensor.shape[0]
    for idx in range(count):
        print(f"i:{idx} to {count}")
        tensor = np.concatenate((tensor, [np.flip(tensor[idx])]), axis=0)
    return tensor
tensor = np.array([[[1, 2] , [3, 4], [5, 6], [7, 8], [9, 10]], [[-1, -2] , [-3, -4], [-5, -6], [-7, -8], [-9, -10]], [[-10, -20] , [-30, -40], [-50, -60], [-70, -80], [-90, -100]]])
print(tensor.shape)
print()
tensor = ArrayExpand(tensor)
print(tensor.shape)
print(tensor)

(3, 5, 2)

i:0 to 3
i:1 to 3
i:2 to 3
(6, 5, 2)
[[[   1    2]
  [   3    4]
  [   5    6]
  [   7    8]
  [   9   10]]

 [[  -1   -2]
  [  -3   -4]
  [  -5   -6]
  [  -7   -8]
  [  -9  -10]]

 [[ -10  -20]
  [ -30  -40]
  [ -50  -60]
  [ -70  -80]
  [ -90 -100]]

 [[  10    9]
  [   8    7]
  [   6    5]
  [   4    3]
  [   2    1]]

 [[ -10   -9]
  [  -8   -7]
  [  -6   -5]
  [  -4   -3]
  [  -2   -1]]

 [[-100  -90]
  [ -80  -70]
  [ -60  -50]
  [ -40  -30]
  [ -20  -10]]]


In [152]:
train_data = ArrayExpand(train_data)
train_labels = ArrayExpand(train_labels)

test_data = ArrayExpand(test_data)
test_labels = ArrayExpand(test_labels)

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

AxisError: axis 2 is out of bounds for array of dimension 2

Contar datos

In [18]:
import numpy as np

def Contar(tensor):
    unique_values, counts = np.unique(tensor, return_counts=True)
    totalSum = np.sum(counts)
    for value, count in zip(unique_values, counts):
        print(f"Número {value}: {count} veces, {(count / totalSum) * 100}")

print(f"Train")
Contar(train_labels[:, 0])

print(f"Test")
Contar(test_labels[:, 0])
    

Train
Número 0.0: 38 veces, 6.551724137931035
Número 1.0: 77 veces, 13.275862068965516
Número 2.0: 29 veces, 5.0
Número 3.0: 436 veces, 75.17241379310344
Test
Número 0.0: 14 veces, 9.929078014184398
Número 1.0: 26 veces, 18.439716312056735
Número 2.0: 13 veces, 9.219858156028367
Número 3.0: 88 veces, 62.4113475177305


Building the Transformer-based model

In [9]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [10]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [11]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = len(np.unique(train_labels[:, 0]))

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = f"Index{INDEX}/_model/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )
    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

In [140]:
trained_model = run_experiment()

Epoch 1/5
31/31 [==============================] - ETA: 0s - loss: 0.9847 - accuracy: 0.6724
Epoch 1: val_loss improved from inf to 0.55275, saving model to tmp\video_classifier
31/31 [==============================] - 7s 209ms/step - loss: 0.9847 - accuracy: 0.6724 - val_loss: 0.5527 - val_accuracy: 0.7701
Epoch 2/5
31/31 [==============================] - ETA: 0s - loss: 0.7169 - accuracy: 0.6866
Epoch 2: val_loss did not improve from 0.55275
31/31 [==============================] - 5s 156ms/step - loss: 0.7169 - accuracy: 0.6866 - val_loss: 0.6198 - val_accuracy: 0.6609
Epoch 3/5
31/31 [==============================] - ETA: 0s - loss: 0.6352 - accuracy: 0.6968
Epoch 3: val_loss did not improve from 0.55275
31/31 [==============================] - 5s 155ms/step - loss: 0.6352 - accuracy: 0.6968 - val_loss: 0.7011 - val_accuracy: 0.7701
Epoch 4/5
31/31 [==============================] - ETA: 0s - loss: 0.6306 - accuracy: 0.7059
Epoch 4: val_loss did not improve from 0.55275
31/31 [==

In [132]:
def prepare_single_video(frames):
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Pad shorter videos.
    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate(frames, padding)

    frames = frames[None, ...]

    # Extract features from the frames of the current video.
    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_features[i, j, :] = 0.0

    return frame_features


def predict_action(dataFrame):
    class_vocab = np.unique(train_labels[:, 0])
    path = dataFrame["Direccion"]
    dimension = dataFrame["Dimension"]
    frames = load_video(path, dimension)
    frame_features = prepare_single_video(frames)
    probabilities = trained_model.predict(frame_features)[0]

    print(class_vocab)
    print(probabilities)
    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

#test_video = test_df.sample(n=1).iloc[0]
test_video = test_df.loc[17]
print(f"Test video path: {test_video}")
test_frames = predict_action(test_video)
#to_gif(test_frames[:MAX_SEQ_LENGTH])

Test video path: Direccion    Noviembre2020\PAC30\PRE.mov
Dimension           [945, 174, 356, 718]
Target                               2.0
Name: 17, dtype: object
1/1 [==============================] - 0s 187ms/step
[0. 1. 2. 3.]
[0.21184069 0.12095597 0.09109778 0.57610554]
  3.0: 57.61%
  0.0: 21.18%
  1.0: 12.10%
  2.0:  9.11%
